In [1]:
import sys

import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.analysis.make_analysis_data as ad
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [ ]:
raw_data = hd.read_raw_sample("777")
df = md.clean_data(raw_data).pipe(md.create_vars).pipe(md.select_sample)
hd.inspect(df)

In [5]:
df = hd.read_sample("777")
hd.inspect(df)

Time for read_sample    : 2.06 seconds
(31,038, 30)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,debit,desco,female,latest_balance,merchant_business_line,postcode,tag_auto,tag_manual,tag_up,ym,yob,balance,income,age,entropy_sptac,region
0,2994252,2012-11-01,14777,12.12,tv licence mbp 6832,tv licensing,spend,communication,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,True,tv licence mbp xxxxxx6832,0.0,729.590027,tv licensing,sw11 5,tv licence,NaN,tv licence,201211,1986.0,-479.021179,14587.92041,35.0,2.649165,London
1,2994250,2012-11-01,14777,900.00,mdbremoved,NaN,NaN,NaN,2013-01-07,2013-01-07,247061,2015-01-13 05:19:00,lloyds bank,current,True,<mdbremoved>,0.0,729.590027,NaN,sw11 5,NaN,rent,rent,201211,1986.0,-479.021179,14587.92041,35.0,2.649165,London


In [301]:
data = ad.main(df)
data

obs   balance_ca   balance_sa  sa_inflows  sa_outflows  \
user_id date                                                                 
14777   2012-11-30   68 -1136.759399  7500.000000         0.0     0.000000   
        2012-12-31   65   349.446136  7500.000000      3000.0     0.000000   
        2013-01-31   64   363.829895  7300.000000       100.0   250.000000   
        2013-02-28   73   393.325714  6422.700195         0.0  1704.599976   
        2013-03-31   72   198.974045  5945.399902       400.0     0.000000   
...                 ...          ...          ...         ...          ...   
538777  2019-08-31  114  2466.928223     0.010000         0.0     0.000000   
        2019-09-30  126  2739.672852     0.010000         0.0     0.000000   
        2019-10-31  131  2356.038574     0.010000         0.0     0.000000   
        2019-11-30  120  2288.697510     0.010000         0.0     0.000000   
        2019-12-31  139  2335.119141     0.010000         0.0     0.000000   

                    sa_net_inflows  sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                                
14777   2012-11-30        0.000000           0.000000            0.000000   
        2012-12-31     3000.000000           2.467795            0.000000   
        2013-01-31     -150.000000           0.071577            0.178943   
        2013-02-28    -1704.599976           0.000000            1.220106   
        2013-03-31      400.000000           0.286309            0.000000   
...                            ...                ...                 ...   
538777  2019-08-31        0.000000           0.000000            0.000000   
        2019-09-30        0.000000           0.000000            0.000000   
        2019-10-31        0.000000           0.000000            0.000000   
        2019-11-30        0.000000           0.000000            0.000000   
        2019-12-31        0.000000           0.000000            0.000000   

                    sa_scaled_net_inflows  log_monthly_spend  \
user_id date                                                   
14777   2012-11-30               0.000000           7.084453   
        2012-12-31               2.467795           6.978186   
        2013-01-31              -0.107366           6.921165   
        2013-02-28              -1.220106           7.288094   
        2013-03-31               0.286309           6.926616   
...                                   ...                ...   
538777  2019-08-31               0.000000           7.210989   
        2019-09-30               0.000000           7.556710   
        2019-10-31               0.000000           6.752586   
        2019-11-30               0.000000           6.894974   
        2019-12-31               0.000000           7.263147   

                    tag_spend_communication  tag_spend_other_transfers  \
user_id date                                                             
14777   2012-11-30                 0.081331                        0.0   
        2012-12-31                 0.109910                        0.0   
        2013-01-31                 0.085427                        0.0   
        2013-02-28                 0.061971                        0.0   
        2013-03-31                 0.094010                        0.0   
...                                     ...                        ...   
538777  2019-08-31                 0.021407                        0.0   
        2019-09-30                 0.015150                        0.0   
        2019-10-31                 0.033856                        0.0   
        2019-11-30                 0.031490                        0.0   
        2019-12-31                 0.006301                        0.0   

                    tag_spend_finance  tag_spend_other_spend  \
user_id date                                                   
14777   2012-11-30           0.035960               0.135259   
        2012-12

In [224]:
def user_period_data(df, user_id, period):
    return (
        df.set_index("user_id", drop=False)
        .loc[user_id]
        .set_index("date", drop=False)
        .loc[period]
        .reset_index(drop=True)
    )

In [303]:
pd.read_csv("/Users/fgu/tmp/data_XX0.csv", nrows=5).columns

Index(['Transaction Reference', 'User Reference', 'User Registration Date',
       'Year of Birth', 'Salary Range', 'Postcode', 'LSOA', 'MSOA',
       'Derived Gender', 'Transaction Date', 'Account Reference',
       'Provider Group Name', 'Account Type', 'Latest Recorded Balance',
       'Transaction Description', 'Credit Debit', 'Amount',
       'User Precedence Tag Name', 'Manual Tag Name', 'Auto Purpose Tag Name',
       'Merchant Name', 'Merchant Business Line', 'Account Created Date',
       'Account Last Refreshed', 'Data Warehouse Date Created',
       'Data Warehouse Date Last Updated', 'Transaction Updated Flag'],
      dtype='object')

In [306]:
fs.ls("3di-data-mdb/raw/")

['3di-data-mdb/raw/',
 '3di-data-mdb/raw/20200630_UserLoginsForNeedham.csv',
 '3di-data-mdb/raw/mdb_000.csv',
 '3di-data-mdb/raw/mdb_000.parquet',
 '3di-data-mdb/raw/mdb_777.csv',
 '3di-data-mdb/raw/mdb_777.parquet',
 '3di-data-mdb/raw/mdb_X77.csv',
 '3di-data-mdb/raw/mdb_X77.parquet',
 '3di-data-mdb/raw/mdb_XX7.csv',
 '3di-data-mdb/raw/mdb_XX7.parquet',
 '3di-data-mdb/raw/mdb_costa.csv']

In [4]:
ha.read_csv("s3://3di-data-mdb/raw/mdb_000.csv", nrows=3, sep="|")

,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,686979264,564777,2020-01-18,1986.0,NaN,CF83 1,W01001427,W02000313,F,2018-10-15,1564118,Lloyds Bank,Current,171.96,nespresso uk ltd,Debit,15.60,"Food, Groceries, Household",No Tag,"Food, Groceries, Household",Nespresso,Nespresso,2020-01-18,2020-04-17 00:10:00,2020-01-19,1900-01-01,C
1,335712120,299777,2015-05-10,1970.0,NaN,BS11 0,E01014498,E02003014,M,2018-04-05,427663,Santander,Current,0.00,"direct debit payment to paypal payment ref 5ae22226hzlca, mandate no 0016",Debit,34.99,Enjoyment,No Tag,Enjoyment,PayPal,PayPal,2015-05-10,2018-09-19 19:58:00,2018-04-07,1900-01-01,C
2,108299228,313777,2015-07-21,1996.0,NaN,KT22 8,E01030553,E02006365,F,2015-12-01,454866,HSBC,Current,75.96,co-op group xx0273 guildford,Debit,13.22,Groceries,No Tag,"Food, Groceries, Household",Co-op,Co-op Supermarket,2015-07-26,2019-08-03 18:22:00,2015-12-02,2018-09-03,U
